In [2]:
import mysql.connector
from functools import partial


def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host='localhost',
        user='megadados',
        password='megadados',
        database='parques',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

In [5]:
db('''
SHOW TABLES
''')

db('''
DESCRIBE TIPOEVENTO
''')

Executando query:
('administradora',)
('evento',)
('parque',)
('tipoevento',)
('usuario',)
Executando query:
('id_tipo', b'int', 'NO', 'PRI', None, '')
('descricao', b'varchar(80)', 'NO', '', None, '')


#### Questão 4a

Escreva uma query que liste todas as administradoras, ordenadas por número de eventos de segurança
ocorridos nos parques que administram. Pegadinha: e se não houve evento de segurança em seus parques?

In [33]:
db('''
SELECT
    ADMINISTRADORA.nome, COUNT(id_evento) as eventos
FROM
    PARQUE
    LEFT OUTER JOIN EVENTO USING (id_parque)
    INNER JOIN ADMINISTRADORA USING (id_adm)
    LEFT OUTER JOIN TIPOEVENTO USING (id_tipo)
WHERE
    TIPOEVENTO.descricao = "Segurança" OR TIPOEVENTO.descricao IS NULL
GROUP BY
    id_adm
ORDER BY
    eventos DESC
''')

Executando query:
('Fundação Fundão', 207)
('Grupo Gripe', 130)
('Sociedade dos Sócios', 105)
('Instituto Instigare', 54)
('Associação Autônoma Andarilha', 0)


#### Questão 4b

Crie uma tabela temporária Total que tem as colunas (id_usuario, total) onde a coluna total indica o
número total de eventos notificados pelo usuário id_usuario.

In [37]:
db('''
CREATE TEMPORARY TABLE Total
    SELECT
        id_usuario, COUNT(id_evento) as total
    FROM
        EVENTO
        INNER JOIN USUARIO USING (id_usuario)
    GROUP BY
        id_usuario
''')

Executando query:


#### Questão 4c

Crie uma tabela temporária chamada UsuarioParque que tem as colunas (id_usuario, id_parque,
contagem) onde contagem é o número de eventos que o usuário id_usuario notificou acerca do parque
id_parque. Dica: GROUP BY permite agrupar por múltiplas colunas.

In [45]:
db('''
CREATE TEMPORARY TABLE UsuarioParque
    SELECT
        id_usuario, id_parque, COUNT(id_evento) as contagem
    FROM
        PARQUE
        INNER JOIN EVENTO USING (id_parque)
    GROUP BY
        id_usuario, id_parque
''')

Executando query:


#### Questão 4d

Um usuário é um fã de um parque se a fração de eventos que ele notifica para o parque, em relação ao número
total de eventos que ele notificou para todos os parques, é superior a 20%. Usando as tabelas temporárias acima
(mesmo que não as tenha desenvolvido) faça uma query que lista os usuários que são fãs de dois ou mais
parques.

In [66]:
db('''
SELECT
    id_usuario, COUNT((contagem*100)/total) as qtd
FROM
    TOTAL
    INNER JOIN USUARIOPARQUE USING (id_usuario)
WHERE
    ((contagem*100)/total) > 20
GROUP BY
    id_usuario
HAVING
    qtd >= 2
''')

Executando query:
(36, 2)
(26, 2)
